In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split

#Plot
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)

testData = pd.read_csv("test.csv")
testY = pd.read_csv("output.csv")
testData['stroke'] = testY['stroke']
trainData = pd.read_csv("train.csv")

#Not Significant for Strokes

trainData = trainData.drop( columns=['ever_married', 'gender', 'id','avg_glucose_level'] )
testData = testData.drop( columns=['ever_married', 'gender', 'id','avg_glucose_level'] )

#Filling Missing BMI Data === train

temp = trainData['bmi'].mean() + trainData['bmi'].median()
temp /= 2
temp
trainData['bmi'] = trainData['bmi'].fillna(temp)

#Filling Missing BMI Data ===test

temp = testData['bmi'].mean() + testData['bmi'].median()
temp /= 2
temp
testData['bmi'] = testData['bmi'].fillna(temp)

# Segregating into Two Models Smoking & Non-Smoking

trainDataNS = trainData[ trainData['smoking_status'].isna() ]
trainDataNS = trainDataNS.drop( columns=['smoking_status'] )
trainDataS = trainData[ trainData['smoking_status'].notna() ]

testDataNS = testData[ testData['smoking_status'].isna() ]
testDataNS = testDataNS.drop( columns=['smoking_status'] )
testDataS = testData[ testData['smoking_status'].notna() ]

# X and Y division

yTrainDataNS = trainDataNS['stroke']
xTrainDataNS = trainDataNS.drop( columns=['stroke'] )

yTrainDataS = trainDataS['stroke']
xTrainDataS = trainDataS.drop( columns=['stroke'] )

yTestDataNS = testDataNS['stroke']
xTestDataNS = testDataNS.drop( columns=['stroke'] )

yTestDataS = testDataS['stroke']
xTestDataS = testDataS.drop( columns=['stroke'] )

#Nominal Categories to one-hot encoding === Train

#Smoke Data
xTrainDataS = pd.get_dummies( xTrainDataS, 
                             columns=[ 'work_type' , 'Residence_type', 'smoking_status']
                            , prefix= ['work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTrainDataNS = pd.get_dummies( xTrainDataNS,
                              columns=[ 'work_type' , 'Residence_type']
                            , prefix=['[work_type' , 'res_type'] )

#Nominal Categories to one-hot encoding ===Test

#Smoke Data
xTestDataS = pd.get_dummies( xTestDataS,
                            columns=['work_type' , 'Residence_type', 'smoking_status']
                            , prefix=['work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTestDataNS = pd.get_dummies( xTestDataNS,
                             columns=['work_type' , 'Residence_type']
                            , prefix=['work_type' , 'res_type'] )

smokeCol = xTrainDataS.columns
nsmokeCol = xTrainDataNS.columns


#Sampling for removing imbalance

from imblearn.over_sampling import SMOTE,RandomOverSampler,ADASYN
smote = SMOTE(random_state=0)

xTrainDataNS, yTrainDataNS = smote.fit_sample(xTrainDataNS, yTrainDataNS)
xTrainDataS, yTrainDataS = smote.fit_sample(xTrainDataS, yTrainDataS)

xTrainDataS = pd.DataFrame( xTrainDataS , columns=smokeCol )
xTrainDataNS = pd.DataFrame( xTrainDataNS , columns=nsmokeCol )
yTrainDataS = pd.DataFrame( yTrainDataS , columns=['stroke'])
yTrainDataNS = pd.DataFrame( yTrainDataNS , columns=['stroke'])

In [3]:
xTrainDataNS.columns

Index(['age', 'hypertension', 'heart_disease', 'bmi', '[work_type_Govt_job',
       '[work_type_Never_worked', '[work_type_Private',
       '[work_type_Self-employed', '[work_type_children', 'res_type_Rural',
       'res_type_Urban'],
      dtype='object')

In [5]:
xTrainDataNS.tail()

,age,hypertension,heart_disease,bmi,[work_type_Govt_job,[work_type_Never_worked,[work_type_Private,[work_type_Self-employed,[work_type_children,res_type_Rural,res_type_Urban
26289,58.216043,0.000000,0.000000,31.168777,0.000000,0.0,1.000000,0.0,0.0,1.000000,0.000000
26290,59.439218,0.000000,0.000000,44.319609,0.000000,0.0,1.000000,0.0,0.0,0.560782,0.439218
26291,80.520617,0.520617,1.000000,23.471137,0.000000,0.0,0.000000,1.0,0.0,0.000000,1.000000
26292,57.455395,0.000000,0.544605,28.152519,0.544605,0.0,0.455395,0.0,0.0,0.000000,1.000000
26293,78.934323,0.000000,1.000000,30.640891,0.000000,0.0,0.000000,1.0,0.0,0.065677,0.934323


In [2]:
from sklearn.model_selection import train_test_split

#Train Splitter

x_train, x_test, y_train, y_test = train_test_split(xTrainDataNS, np.ravel( yTrainDataNS , order='C' ) 
                                                    , test_size=0.30, random_state=0)

x_trainS, x_testS, y_trainS, y_testS = train_test_split(xTrainDataS, np.ravel( yTrainDataS , order='C' ),
                                                        test_size=0.30, random_state=0)

In [3]:
from sklearn.preprocessing import StandardScaler
import sys

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
xTestDataNS = scaler.transform( xTestDataNS )



scaler = StandardScaler()
scaler.fit(x_trainS)

x_trainS = scaler.transform(x_trainS)
x_testS = scaler.transform(x_testS)
xTestDataS = scaler.transform( xTestDataS )


/home/ncheck/anaconda3/envs/deepenv/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  
/home/ncheck/anaconda3/envs/deepenv/lib/python3.6/site-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


In [ ]:
#MLP

from sklearn.neural_network import MLPClassifier

alpha=0.00002
for i in range(18):
    
    alpha += 0.00002
    sModel = MLPClassifier( hidden_layer_sizes=(64,64,64) , max_iter=300 , alpha=alpha  )


    sModel.fit(x_trainS, y_trainS)


    predictions = sModel.predict(x_testS)

    print(sModel.score(x_testS, y_testS))
    print(metrics.confusion_matrix(y_testS, predictions))


    #Blind Test

    predictions = sModel.predict(xTestDataS)


    print(sModel.score(xTestDataS, yTestDataS))
    print(metrics.confusion_matrix(yTestDataS, predictions))
    
    print("*************", alpha)



0.938355389661803
[[8231  549]
 [ 541 8361]]
0.701556420233463
[[8818  625]
 [3210  197]]
************* 4e-05
0.9357538739961543
[[8308  472]
 [ 664 8238]]
0.705603112840467
[[8911  532]
 [3251  156]]
************* 6.000000000000001e-05
0.9350186630471666
[[8178  602]
 [ 547 8355]]
0.7007003891050584
[[8797  646]
 [3200  207]]
************* 8e-05


In [ ]:

"""
0.7962576954772408
[[7369 2899]
 [1304 9057]]
0.5964980544747082
[[6704 2739]
 [2446  961]]

$ hidden_layer_sizes=(16,64,64,64) , max_iter=300 , alpha=0.00015 

0.9261234184885355
[[ 8849  1419]
 [  105 10256]]
0.6644357976653696
[[8089 1354]
 [2958  449]]

$ dropped glucose hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.00015

0.8628144844636192
[[8092 2176]
 [ 654 9707]]
0.6327626459143969
[[7447 1996]
 [2723  684]]

$ hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.0003

0.8783266275631393
[[ 8073  2195]
 [  315 10046]]
0.6292607003891051
[[7398 2045]
 [2719  688]]

$ hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.0006

0.8612632701536672
[[7882 2386]
 [ 476 9885]]
0.6214007782101167
[[7214 2229]
 [2636  771]]

$ hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.0006 scaled

0.9007222841630714
[[9067 1201]
 [ 847 9514]]
0.674863813229572
[[8312 1131]
 [3047  360]]

$ hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.0001 scaled

0.9171069853119395
[[ 8657  1611]
 [   99 10262]]
0.6598443579766538
[[7974 1469]
 [2902  505]]

$ hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.00005 scaled

0.9002375296912114
[[ 8299  1969]
 [   89 10272]]
0.6368093385214008
[[7576 1867]
 [2800  607]]

$ hidden_layer_sizes=(16,64,64) , max_iter=300 , alpha=0.00005  smote

0.9247257097613392
[[8210  570]
 [ 761 8141]]
0.7001556420233463
[[8789  654]
 [3199  208]]


"""

In [10]:
from joblib import dump

dump( sModel , 'smokemlp.joblib' )

['smokemlp.joblib']

In [13]:

model = MLPClassifier(alpha=0.0002, max_iter=400 )


model.fit(x_train, y_train)


predictions = model.predict(x_test)

print(model.score(x_test, y_test))
print(metrics.confusion_matrix(y_test, predictions))

predictions = model.predict(xTestDataNS)


print(model.score(xTestDataNS, yTestDataNS))
print(metrics.confusion_matrix(yTestDataNS, predictions))


"""
0.8876453330435727
[[3606  972]
 [  62 4563]]
0.6395409494001043
[[3340  867]
 [1206  338]]

"""

0.8876453330435727
[[3606  972]
 [  62 4563]]
0.6395409494001043
[[3340  867]
 [1206  338]]


In [14]:
dump( model , 'nosmokemlp.joblib' )

['nosmokemlp.joblib']

In [36]:
from collections import Counter

Counter(y_testS)

Counter({1: 8902, 0: 8780})